In [33]:
from scipy.io import loadmat
import networkx as nx
from util_functions import sample_train_val_test
import _pickle as cPickle
import random
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix

In [13]:
data = loadmat("./data/cora.mat")
label = data['label'] #获取label
net_c = data['net']   # input: clean network (not consider missing links) 无噪声边
print(net_c.shape)


(2708, 2708)


In [14]:
data = loadmat("./data/cora_n0.1.mat")
net = data['net']   # input: flawed network 缺陷网络
node_num = net.shape[0] #获取节点数量
print(net.shape)


(2708, 2708)


In [16]:
edges = nx.from_scipy_sparse_array(net).edges() #带噪声的边
edges_clean = nx.from_scipy_sparse_array(net_c).edges() #无噪声的边

print(f'# of nodes: {node_num}')
print(f'# of clean/all edges: {len(edges_clean)}/{len(edges)}')
num_noisy_edge = len(edges) - len(edges_clean) #获取噪声边的数量

# of nodes: 2708
# of clean/all edges: 5278/5805


In [19]:
#节点属性
if  'group' in data:
    # load node attributes (here a.k.a. node classes)
    try:
        attributes = data['group'].toarray().astype('float32')
    except AttributeError:
        attributes = data['group'].astype('float32')
else:
    attributes = None
print('attribute dimension: {}'.format(attributes.shape))

attribute dimension: (2708, 1433)


In [29]:
#Sample train and test links
split_ratio=[8,1,1]#80%训练,10%验证,10%测试
missing_ratio=0.1
neg_pos_ratio=5
train_val_test, missing_links = sample_train_val_test(edges, edges_clean, net,split_ratio,missing_ratio)
#pos:真正的缺失链路
#neg: 假的缺失链路，包括两种情况: 1.噪声链路 2.链路本身不存在
print('MISSING LINK PREDICTION-- # train_pos: %d, # train_neg: %d, # val_pos: %d, # val_neg: %d, # test_pos: %d, # test_neg: %d' % (
    len(train_val_test['train'][0][0]), len(train_val_test['train'][1][0]), len(train_val_test['val'][0][0]),
    len(train_val_test['val'][1][0]), len(train_val_test['test'][0][0]), len(train_val_test['test'][1][0])))


MISSING LINK PREDICTION-- # train_pos: 428, # train_neg: 428, # val_pos: 53, # val_neg: 265, # test_pos: 53, # test_neg: 265


In [30]:
A = net.copy()  # the observed network
# mask missing links
for key, value in train_val_test.items():#在A的测试集上测试
    A[value[0][0], value[0][1]] = 0
    A[value[0][1], value[0][0]] = 0


In [35]:
neg_pos_ratio=5
pos_noisy_links = [e for e in edges if e not in edges_clean and e[::-1] not in edges_clean]
remain_clean_links = [e for e in edges_clean if e not in missing_links and e[::-1] not in missing_links]#既不是缺失边也不是噪声边
perm = random.sample(range(len(remain_clean_links)), len(pos_noisy_links) * neg_pos_ratio)
neg_noisy_links = [remain_clean_links[i] for i in perm]
noisy_candidate = pos_noisy_links + neg_noisy_links
row = np.array(noisy_candidate)[:,0]; col = np.array(noisy_candidate)[:,1]
data = [1 for i in range(len(row))]
Identity = csc_matrix((data, (row,col)), shape=A.shape)